In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

# Set up the WebDriver
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

# Open the target webpage
url = "https://www.binance.com/en/square/news/all?ref=WMNC7PBZ&gad_source=1&gclid=CjwKCAiAk8G9BhA0EiwAOQxmfm5p9TwWbHe31bpJ9ZOUuvcgqi1mT59U3oKgxM5L65oM10f6KJdqcRoCVhYQAvD_BwE&ads=true&utm_source=googleadwords_int&utm_medium=cpc"
driver.get(url)

# Wait for the main container to be available
WebDriverWait(driver, 0.5).until(EC.presence_of_element_located((By.CLASS_NAME, "css-mycpt4")))

# Loop to scroll down and scrape content
while True:
    # Locate all news items inside the main container
    folder = driver.find_element(By.CLASS_NAME, "css-mycpt4")
    items = folder.find_elements(By.CLASS_NAME, "css-vurnku")

    for item in items:
        try:
            time_info = item.find_element(By.CLASS_NAME, "css-vyak18").text if item.find_elements(By.CLASS_NAME, "css-vyak18") else "N/A"
            
            # Find the title inside <a> tag
            title_element = item.find_element(By.TAG_NAME, "a") if item.find_elements(By.TAG_NAME, "a") else None
            title = title_element.text if title_element else "N/A"

            # Find the trading pair inside <span class="symbol">
            trading_pair_element = item.find_element(By.CLASS_NAME, "symbol") if item.find_elements(By.CLASS_NAME, "symbol") else None
            trading_pair = trading_pair_element.text if trading_pair_element else "N/A"

            # Find price change inside <span class="index price-decline">
            price_change_elements = item.find_elements(By.CSS_SELECTOR, ".index, .price-decline, .price-raise")

            price_change = price_change_elements[0].text if price_change_elements else "N/A"

            with open("scraped_data.txt", "a", encoding="utf-8") as file:
                file.write(f"Time: {time_info}\n")
                file.write(f"Article Title: {title}\n")
                file.write(f"Trading Pair: {trading_pair}\n")
                file.write(f"Price Change: {price_change}\n")
                file.write("=" * 50 + "\n")

        except Exception as e:
            print(f"Skipping an item due to error: {e}")

    # Check if we've reached the end of the page, if not, scroll down
    try:
        # Scroll down to load more items
        driver.execute_script("window.scrollBy(0, 3000);")
        time.sleep(0.5)  # Wait for new elements to load

        # Check if there are more items, if not, break the loop
        new_items = driver.find_elements(By.CLASS_NAME, "css-vurnku")
        if len(new_items) == len(items):  # No new items loaded
            break
        items = new_items  # Update items list with the newly loaded items

    except Exception as e:
        print(f"Error while scrolling: {e}")
        break



print("Scraping complete! Data saved in 'scraped_data.txt'.")


# Close browser
driver.quit()


Skipping an item due to error: ('Connection aborted.', ConnectionResetError(10054, 'Une connexion existante a dû être fermée par l’hôte distant', None, 10054, None))


KeyboardInterrupt: 

In [3]:
def clean_text_file(input_file, output_file):
    with open(input_file, 'r', encoding='utf-8') as file:
        lines = file.read().split('==================================================\n')
    
    cleaned_entries = set()
    valid_entries = []
    
    for entry in lines:
        entry = entry.strip()
        if entry and "Time: N/A" not in entry:
            if entry not in cleaned_entries:
                cleaned_entries.add(entry)
                valid_entries.append(entry)
    
    with open(output_file, 'w', encoding='utf-8') as file:
        file.write("\n==================================================\n".join(valid_entries))
    
# Usage example:
clean_text_file('scraped_data.txt', 'scraped_data_clean.txt')